In [1]:
import sys
import os 
os.chdir('..')
sys.path.append("./scripts")
import numpy as np
import config
import torch
import data_collection.BLong_preamble_detection as pd
import utils
from scramble_table import *
import DANF.blong_dnn as dnn
import raw_data_preprocessing as rdp
import opt.goodput_opt as go
from datetime import datetime
f = None

cur_data = None
# device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
power_of_2 = np.array([1, 2, 4, 8, 16, 32, 64, 128], dtype=np.uint8)
target_aa_bit = np.array([0,0,0,1,1,0,1,1,0,1,1,1,1,1,0,1,1,0,0,1,0,0,0,1,0,1,1,1,0,0,0,1], dtype=np.uint8)
preamble_bits = np.array([0, 1, 0, 1, 0, 1, 0, 1], dtype=np.uint8)
header_bits = np.array([0,0,0,0,1,1,1,0], dtype=np.uint8)
len_bits = np.array([1,1,1,1,1,1,1,1], dtype=np.uint8)

In [3]:
dnn_cpy_wr_best_round = {1: 5500, 2:5600, 4:5300, 8:4700, 16:5100, 32:4000, 64:4900}
def case_study_process(raw_file, extf, num):
    if extf > 1:
        data_bits = np.concatenate([target_aa_bit, header_bits, rand_bits])
        data_bits = data_bits[:(num-1)*8]
    else:
        data_bytes = np.concatenate([np.array([0x70, 0xff], dtype=np.uint8), scramble_table[0, :255]])
        data_bytes = data_bytes ^ scramble_table[12, :257]
        data_bytes = np.concatenate([data_bytes, np.array([224, 247, 242], dtype=np.uint8)])
        data_bits = utils.bytes_to_bits(data_bytes)
        data_bits = np.concatenate([target_aa_bit, data_bits])
    ############################
    raw_data = np.load("./raw_data/" + raw_file.format("", extf) + ".npz")
    raw_signal_ori = raw_data["arr_0"]
    raw_signal_ori = raw_signal_ori.flatten()[np.newaxis, :]
    raw_signal = utils.filter(raw_signal_ori)
    thres = 50
    p_idx, stft_p_idx, det_num = pd.preamble_detection_aa_check(raw_signal_ori, raw_signal, extf, thres_percent=thres, full=True, getdet=True, model_name="dnn_uni_cpy_wr_dis_{}e_{}s.pt".format(extf, dnn_cpy_wr_best_round[extf]))
    p_idx = p_idx[0]
    det_num = det_num[0]
    stft_p_idx = stft_p_idx[0]
    # Then we start to slice the data bytes
    if extf > 1:
        bits_pre_pdu = int(2040 // extf)
    else:
        bits_pre_pdu = 265 * 8
    res_len = 2040 - bits_pre_pdu * extf
    skip_len = (80 + 150 + res_len) * config.sample_pre_symbol
    packets = []
    done_flag = False
    print("has {} packets".format(len(p_idx)))
    for i in range(len(p_idx)):
        pkt = []
        ptr = 8 * extf * config.sample_pre_symbol + p_idx[i]
        for j in range(8, num*8):
            if j % bits_pre_pdu == 0:
                ptr += skip_len
            n_ptr = ptr + extf * config.sample_pre_symbol
            sp = ptr - config.extra
            ep = n_ptr + config.extra
            if ep >= raw_signal_ori.shape[1]:
                done_flag = True
            pkt.append(raw_signal_ori[0, sp:ep])
            ptr = n_ptr
        if done_flag:
            break
        packets.append(pkt)
    packets = np.array(packets)
    if packets.shape[0] == 0:
        return np.zeros(3), np.zeros(3), 1
    stft_packets = []
    stft_done_flag = False
    for i in range(len(stft_p_idx)):
        pkt = []
        ptr = 8 * extf * config.sample_pre_symbol + stft_p_idx[i]
        for j in range(8, num*8):
            if j % bits_pre_pdu == 0:
                ptr += skip_len
            n_ptr = ptr + extf * config.sample_pre_symbol
            sp = ptr - config.extra
            ep = n_ptr + config.extra
            if ep >= raw_signal_ori.shape[1]:
                stft_done_flag = True
                break
            pkt.append(raw_signal_ori[0, sp:ep])
            ptr = n_ptr
        if stft_done_flag:
            break
        stft_packets.append(pkt)
    stft_packets = np.array(stft_packets)
    print(stft_packets.shape)
    np.savez("./processed_data/" + raw_file.format("_sensys", "pkt_cpy"), packets=stft_packets, data_bits=data_bits, stft_flag=done_flag)
    np.savez("./processed_data/" + raw_file.format("_sensys", "blongpkt_cpy"), packets=packets, data_bits=data_bits, done_flag=done_flag)
    
    # temp_d = np.load("./processed_data/" + raw_file.format("pkt") + ".npz")
    # det_num = 0
    # packets = temp_d["packets"]
    
    labels = np.concatenate([data_bits[:(num-1)*8] for _ in range(packets.shape[0])])
    bcrs = []
    packets = packets.reshape([-1, packets.shape[2]])
# 2, 4, 8, 16, 32, 64
    for dextf in [1, 2, 4, 8, 16, 32, 64]:
        print("dext", dextf, end=" ")
        dext_syms, _, _, dext_labels = rdp.save_dext_slices(packets, None, None, labels, extf, dextf, raw_file.format("_sensys/", dextf) + "_cpy", 1)
        model_name = "dnn_uni_cpy_wr_dis_{}e_{}s.pt".format(dextf, dnn_cpy_wr_best_round[dextf])
        bits, correct_num = dnn.decode_ber(dext_syms, dext_labels, dextf, model_name)
        bcr = correct_num / dext_syms.shape[0]
        bcrs.append(bcr)
        print(bcr)
        # print(raw_file, dextf, bcrs[-1])
    return bcrs, det_num

def get_detection_rate(file, extf):
    data = np.load("./raw_data/" + file + ".npz")
    raw_signal_ori = data["arr_0"].flatten()[np.newaxis, :]
    raw_signal = utils.filter(raw_signal_ori)
    thres = 50
    pkt_detect_rates = pd.preamble_detection_aa_check_test(raw_signal_ori, raw_signal, extf, thres_percent=thres, full=True, model_name="dnn_uni_cpy_wr_dis_{}e_{}s.pt".format(extf, dnn_cpy_wr_best_round[extf]))[0]
    if extf < 2:
        return pkt_detect_rates[1] / 1024
    else:
        return pkt_detect_rates[0] / 1024

def get_detection_rate_all(p):
    rates = []
    for extf in [1, 2, 4, 8, 16, 32, 64]:
        if p in ["H", "G", "I"]:
            r = get_detection_rate("./case_study/nlos/{}pos_12c_{}e_preamble_0".format(p, extf), extf)
        else:
            r = get_detection_rate("./case_study/nlos/{}pos_12c_{}e_preamble".format(p, extf), extf)
        rates.append(r)
    return rates

def round_up(data, nums=4):
    scale = 10 ** nums
    rdata = data * scale
    rdata = np.ceil(rdata)
    rdata /= scale
    rdata[np.where(rdata>1)] = 1
    return rdata

def raw_data_processing(p, pylen=265, load=False):
    print("Process", p)
    if not load:
        all_bcrs, det_num = case_study_process("case_study/" + "nlos{}" + "/{}pos_12c_{}e_{}n_{}".format(p, "{}", pylen, 0), 64, 265)
        bcrs = [all_bcrs[0][1], all_bcrs[1][1]]
        for i in range(len(all_bcrs)-2):
            bcrs.append(all_bcrs[i+2][0])
        bcrs = np.array(bcrs)
        print("bcrs", bcrs)
        print("detection_num", det_num)
        
        # Then get the detection rate
        pdetr = get_detection_rate_all(p)
        pdetr = np.array(pdetr)
        print("packet detection rate", pdetr)
        np.savez("./output/nlos_opt_sensys/cpy_{}_pos.npz".format(p), bcrs=np.array(bcrs), pdetr=np.array(pdetr), det_num=det_num)
    else:
        data = np.load("./output/nlos_opt_sensys/cpy_{}_pos.npz".format(p))
        bcrs = data["bcrs"]
        pdetr = data["pdetr"]
        print(bcrs)
        print(pdetr)
    bcrsb = bcrs
    pdetrb = pdetr
    # because of the boundary issue, the actual BCR could be little bit higher than measured
    bcrs = round_up(bcrs, 4)
    pdetr = round_up(pdetr, 4)
    print(bcrs)
    print(pdetr)
    payload_n_max, extf, goodput = go.goodput_optimiation_nlos_fixn(bcrs, pdetr)
    
    print(p, "->", payload_n_max+10, extf)
    extf_idx = int(np.log2(extf))
    print("tab:{} {} {} {:.2f}\% {:.2f}\%".format(p, extf, payload_n_max, pdetrb[extf_idx]*100, (1-bcrsb[extf_idx])*100))
    return payload_n_max+10, payload_n_max, extf

def max_throughput(extf, payload_n_max):
    if extf == 1:
        mt = payload_n_max / (payload_n_max + 10 + 150 / 8)
    else:
        n_total = (payload_n_max + 10) * extf
        n_bit = np.floor(255 * 8 / extf)
        n_aip = np.ceil(n_total / n_bit)
        mt = payload_n_max / (n_total + (10 + 150 / 8) * n_aip)
    # unit bit / us or Mb/s
    return mt
        
def get_real_throughput(p, n_max, extf, all_num=240, load=False, nth=0):
    if extf > 1:
        data_bits = np.concatenate([target_aa_bit, header_bits, rand_bits])
    else:
        data_bytes = np.concatenate([np.array([0x70, 0xff], dtype=np.uint8), scramble_table[0, :255]])
        data_bytes = data_bytes ^ scramble_table[12, :257]
        data_bytes = np.concatenate([data_bytes, np.array([28, 134, 81], dtype=np.uint8)])
        data_bits = utils.bytes_to_bits(data_bytes)
        data_bits = np.concatenate([target_aa_bit, data_bits])
    data_bits = data_bits[:(n_max-1)*8]
    
    # if extension is 1 or 64, we can reuse the dataset to test the native
    if extf !=64 and not load:
        # F: n=1 H: n=1, G
        raw_data = np.load("./raw_data/case_study/nlos/{}pos_12c_{}e_{}n_{}.npz".format(p, int(extf), n_max, nth))
        raw_signal_ori = raw_data["arr_0"].flatten()[np.newaxis, :]
        raw_signal = utils.filter(raw_signal_ori)
        thres = 50
        p_idx, stft_p_idx, _ = pd.preamble_detection_aa_check(raw_signal_ori, raw_signal, extf, thres_percent=thres, full=True, getdet=True,model_name="dnn_uni_cpy_wr_dis_{}e_{}s.pt".format(extf, dnn_cpy_wr_best_round[extf]))
        if extf < 2:
            p_idx = stft_p_idx[0]
        else:
            p_idx = p_idx[0]
        # Then we start to slice the data bytes
        if extf > 1:
            bits_pre_pdu = int(2040 // extf)
        else:
            bits_pre_pdu = 265 * 8
        res_len = 2040 - bits_pre_pdu * extf
        skip_len = (80 + 150 + res_len) * config.sample_pre_symbol
        packets = []
        done_flag = False
        print("has {} packets".format(len(p_idx)))
        for i in range(len(p_idx)):
            pkt = []
            ptr = 8 * extf * config.sample_pre_symbol + p_idx[i]
            for j in range(8, n_max*8):
                if j % bits_pre_pdu == 0:
                    ptr += skip_len
                n_ptr = ptr + extf * config.sample_pre_symbol
                sp = ptr - config.extra
                ep = n_ptr + config.extra
                if ep >= raw_signal_ori.shape[1]:
                    done_flag = True
                    break
                pkt.append(raw_signal_ori[0, sp:ep])
                ptr = n_ptr
            if done_flag:
                break
            packets.append(pkt)
        packets = np.array(packets)
        np.savez("./processed_data/case_study/nlos_sensys/cpy_{}pos_12c_{}e_{}n_blong.npz".format(p, int(extf), n_max), packets=packets)
    elif extf == 64:
        temp_d = np.load("./processed_data/case_study/nlos_sensys/cpy_{}pos_12c_{}e_{}n_{}.npz".format(p, "blongpkt", 265, 0))
        packets = temp_d["packets"]
        packets = packets[:, :, :(n_max-1)*8]
    else:
        temp_d = np.load("./processed_data/case_study/nlos_sensys/cpy_{}pos_12c_{}e_{}n_blong.npz".format(p, int(extf), n_max))
        packets = temp_d["packets"]
    if len(packets) == 0:
        return 0, 0
    _, _, dnn_correct_len, stft_correct_len = dnn.dnn_decode_throghput(packets.reshape([-1, packets.shape[2]]), extf, data_bits, model_name="dnn_uni_cpy_wr_dis_{}e_{}s.pt".format(extf, dnn_cpy_wr_best_round[extf]))
    if extf < 2:
        return packets.shape[0] / all_num, stft_correct_len[0] / stft_correct_len[1]
    else:
        return packets.shape[0] / all_num, dnn_correct_len[0] / dnn_correct_len[1]


best_nth = {"C": 0, "D": 0, "F": 2, "H":0, "G": 0, "I": 0}
#  "F", "C", "D", "H",  "I"
for p in ["H", "F", "G", "C", "D",  "I"]:
    n_max, payload_n_max, extf = raw_data_processing(p, load=True)
    print(p, extf)
    t = get_real_throughput(p, 265, extf, nth=best_nth[p], load=True)
    blong_th = 1000 * max_throughput(extf, 255) * t[0] * t[1]
    print("throughput:", p, blong_th, "pdetr", t[0], "ber", 1-t[1])   

Process H
[0.9815   0.990815 0.994765 0.995905 0.99661  0.99704  0.997435]
[0.24316406 0.74511719 0.93164062 0.96777344 0.98828125 0.99707031
 0.99609375]
[0.9815 0.9909 0.9948 0.996  0.9967 0.9971 0.9975]
[0.2432 0.7452 0.9317 0.9678 0.9883 0.9971 0.9961]
H -> 265 2
tab:H 2 255 74.51\% 0.92\%
H 2


H tensor(405.5170) pdetr 0.8916666666666667 ber tensor(0.0035)
Process F
[0.90854  0.944595 0.985715 0.996065 0.997935 0.998805 0.99923 ]
[0.         0.01367188 0.38574219 0.61621094 0.86914062 0.97851562
 0.98144531]
[0.9086 0.9446 0.9858 0.9961 0.998  0.9989 0.9993]
[0.     0.0137 0.3858 0.6163 0.8692 0.9786 0.9815]
F -> 265 4
tab:F 4 255 38.57\% 1.43\%
F 4
F tensor(134.8608) pdetr 0.6208333333333333 ber tensor(0.0236)
Process G
[0.66566  0.701205 0.785555 0.86894  0.934065 0.968905 0.98677 ]
[0.01367188 0.00488281 0.23046875 0.65820312 0.78125    0.90722656
 0.95898438]
[0.6657 0.7013 0.7856 0.869  0.9341 0.969  0.9868]
[0.0137 0.0049 0.2305 0.6583 0.7813 0.9073 0.959 ]
G -> 265 8
tab:G 8 255 65.82\% 13.11\%
G 8
G tensor(66.8599) pdetr 0.6375 ber tensor(0.0217)
Process C
[0.99287  0.99744  0.998655 0.999185 0.999615 0.999625 0.999775]
[0.984375   0.97460938 0.99707031 1.         0.99609375 0.99707031
 1.        ]
[0.9929 0.9975 0.9987 0.9992 0.9997 0.9997 0.9998]
[0.9844 0.9747 0.99

Then, test for the native BLE and Symphone

In [ ]:
def max_throughput(extf, payload_n_max):
    if extf == 1:
        mt = payload_n_max / (payload_n_max + 10 + 150 / 8)
    else:
        n_total = (payload_n_max + 10) * extf
        n_bit = np.floor(255 * 8 / extf)
        n_aip = np.ceil(n_total / n_bit)
        mt = payload_n_max / (n_total + (10 + 150 / 8) * n_aip)
    # unit bit / us or Mb/s
    return mt

def get_symphony_throughput(p, all_num=240, pdu_len=39):
    data = np.load("./processed_data/case_study/nlos_sensys/{}pos_12c_pkt_cpye_265n_0.npz".format(p))
    pkt = data["packets"]
    data_bits = data["data_bits"]
    done_flag = data["stft_flag"]
    print(pkt.shape)
    print(data_bits.shape)
    packets = pkt[:, :(pdu_len+9)*8, :].copy()
    valid_idx, corr_num = dnn.stft_decode_bits(packets.reshape([-1, packets.shape[2]]), 64, data_bits[:(pdu_len+9)*8])
    if done_flag:  # done flag 表明最后一个包检测到了，但是长度不够
        all_num -= 1
    return packets.shape[0] / all_num, corr_num[0] / corr_num[1]

def get_native_throughput(p, all_num=240, pdu_len=39, load=False):
    data_bytes = np.concatenate([np.array([0x70, 0xff], dtype=np.uint8), scramble_table[0, :255]])
    data_bytes = data_bytes ^ scramble_table[12, :257]
    # crc_bytes = np.array([224, 247, 242], dtype=np.uint8) ^ np.array([59, 252, 113], dtype=np.uint8) ^ np.array([199, 141, 210], dtype=np.uint8)
    crc_bytes = np.array([28, 134, 81], dtype=np.uint8)  # The crc bytes with scramble (chan = 12)
    # print(crc_bytes)
    data_bytes = np.concatenate([data_bytes, crc_bytes])
    data_bits = utils.bytes_to_bits(data_bytes)
    data_bits = np.concatenate([target_aa_bit, data_bits])
    
    if not load:
        raw_data = np.load("./raw_data/case_study/nlos/{}pos_12c_1e_265n_0.npz".format(p))
        raw_signal_ori = raw_data["arr_0"].flatten()[np.newaxis, :]
        raw_signal = utils.filter(raw_signal_ori)
        thres = 50
        p_idx = pd.preamble_detection_aa_check_native(raw_signal_ori, raw_signal, 1, thres_percent=thres, full=True)
        p_idx = p_idx[0]
        packets = []
        done_flag = False
        print("has {} native packets".format(len(p_idx)))
        for i in range(len(p_idx)):
            pkt = []  
            ptr = 8 * 1 * config.sample_pre_symbol + p_idx[i]
            for j in range(8, 265*8):
                n_ptr = ptr + 1 * config.sample_pre_symbol
                sp = ptr - config.extra
                ep = n_ptr + config.extra
                if ep >= raw_signal_ori.shape[1]:
                    done_flag = True
                    if i != len(p_idx):
                        raise Exception("No long enouth for continous packets")
                    break
                pkt.append(raw_signal_ori[0, sp:ep])
                ptr = n_ptr
            if done_flag:
                break
            packets.append(pkt)
        packets = np.array(packets)
        np.savez("./processed_data/case_study/nlos_sensys/native_{}pos_12c_1e_265n.npz".format(p), packets=packets, done_flag=done_flag)
        if len(p_idx) == 0:
            return 0, 0
    else:
        temp_d = np.load("./processed_data/case_study/nlos_sensys/native_{}pos_12c_1e_265n.npz".format(p))
        packets = temp_d["packets"]
        print(packets.shape)
        done_flag = temp_d["done_flag"]
        if len(packets) == 0:
            return 0, 0

    # packets = np.load("./processed_data/case_study/nlos_sensys/{}pos_12c_1e_265n.npz".format(p))["packets"]
    detection_num = packets.shape[0]
    packets = packets[:, :(pdu_len+9)*8, :].copy()
    _, valid_idx, corr_num = dnn.native_decode_bits(packets.reshape([-1, packets.shape[2]]), 1, data_bits[:(pdu_len+9)*8])
    if done_flag:
        all_num -= 1
    return detection_num / all_num, corr_num[0] / corr_num[1]

with open("./output/nlos_opt/symphony_goodput.log", "w") as f:
    for p in ["F", "G", "C", "D", "H",  "I"]:
        t = get_symphony_throughput(p, pdu_len=255)
        stft_th = 1000 * max_throughput(64, 255) * t[0] * t[1]
        print(p, stft_th)
        f.write("{}, {}\n".format(p, stft_th))
with open("./output/nlos_opt/native_goodput.log", "a") as f:
    currentTime = datetime.now().strftime("%D   %H:%M:%S")
    f.write(currentTime + "\n")
    for p in ["F", "G", "C", "D", "H",  "I"]:
        t = get_native_throughput(p, pdu_len=255, load=True)
        print(t)
        native_th = 1000 * max_throughput(1, 255) * t[0] * t[1]
        print(p, native_th)
        f.write("{}, {}\n".format(p, native_th))


(233, 2112, 528)
(2112,)
F tensor(7.5872)
(121, 2112, 528)
(2112,)
G tensor(3.8584)
(239, 2112, 528)
(2112,)
C tensor(7.7921)
(168, 2112, 528)
(2112,)
D tensor(5.4625)
(231, 2112, 528)
(2112,)
H tensor(7.5099)
(224, 2112, 528)
(2112,)
I tensor(7.2372)
preamble conv  0.3885200023651123
peak detection 3.4793202877044678
pa conv 2.852829933166504
109059 possible preamble
[ 11125  12856  15690  21892  23693  27442 108297]
has 7 native packets
(0.029166666666666667, tensor(0.9332))
F tensor(24.4615)
preamble conv  0.40714025497436523
peak detection 3.1817171573638916
pa conv 3.014329671859741
124521 possible preamble
[]
has 0 native packets
(0, 0)
G 0.0
preamble conv  0.4143211841583252
peak detection 2.629829168319702
pa conv 2.778358221054077
108672 possible preamble
[   435    869   1321   1726   2153   2572   3035   3496   3954   4384
   4842   5321   5801   6263   6747   7228   7650   8090   8532   9132
   9588  10062  10505  10936  11390  11965  12458  12952  13578  14041
  14574  150

In [6]:
# Compare the throughput
    #   \textbf{A}        & 894.89                  & 7.79              & 894.88              \\ \hline
    #   \textbf{B}        & 699.58                  & 5.46              & 676.17              \\ \hline
    #   \textbf{C}        & 134.86                  & 7.59              & 24.46               \\ \hline
    #   \textbf{D}        & 66.86                   & 3.86              & 0.00                \\ \hline
    #   \textbf{E}        & 405.52                  & 7.51              & 248.13               \\ \hline
    #   \textbf{F}        & 824.22                  & 7.24              & 812.53              \\ \hline
symphony = np.array([7.79, 5.46, 7.59, 3.86, 7.51, 7.24])
native = np.array([894.88, 676.17, 24.46, 0.00, 248.13, 812.53])
blew = np.array([894.89, 699.58, 134.86, 66.86, 405.52, 824.22])
a = blew - native
b = blew - symphony
print([np.min(a), np.max(a)])
print([np.min(b), np.max(b)])

[0.009999999999990905, 157.39]
[63.0, 887.1]
